# AutoML Tabular ile Yorum Sınıflandırma (Text Column) — Predefined Split Fix

Hata: **`Val split contains no sample`**  
Sebep: Demo dataset çok küçük olduğunda `validation_fraction_split` ile otomatik split bazen validation setine 0 satır düşürebilir.

Çözüm: CSV’ye bir `split` kolonu ekleyip **predefined split** kullanmak.  
`split` değerleri: `TRAIN`, `VALIDATE`, `TEST`.


## 0) Kurulum & Auth

In [ ]:
%pip -q install google-cloud-aiplatform==1.* google-cloud-storage==2.* pandas==2.*

# Kurulum sonrası kernel restart gerekebilir.


In [ ]:
import os
# ADC:
#   gcloud auth application-default login
# Service account:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/key.json"
print("Auth hazır.")


## 1) Konfigürasyon

In [ ]:
from google.cloud import aiplatform

project_id  = "vertextraining-486212"
region      = "europe-west2"      # dataset + training + endpoint aynı region
bucket_name = "my-vertex-training-bucket"  # örn: my-vertex-bucket
gcs_prefix  = "automl_tabular_text_demo"

dataset_display_name  = "tabular-text-demo-reviews"
model_display_name    = "tabular-text-demo-model"
endpoint_display_name = "tabular-text-demo-endpoint"

target_column = "label"
text_feature_column = "review_text"
split_column = "split"

aiplatform.init(project=project_id, location=region, staging_bucket=bucket_name)
print("✅ init OK")


## 2) Örnek veri üret + split kolonu

Her label için 5 örnek var →
- 3 TRAIN
- 1 VALIDATE
- 1 TEST

Böylece validation seti asla 0 olmaz ve sınıflar dengeli kalır.


In [ ]:
import random
import pandas as pd

random.seed(42)

pos_templates = [
    "Kargo {hiz} geldi, ürün {kalite}. {tavsiye}",
    "Paketleme {paket}, ürün {kalite}. {memnuniyet}",
    "{performans} ve {fiyatperf}. {tavsiye}",
    "Kurulum {kolay}, kullanım {rahat}. {memnuniyet}",
]
neg_templates = [
    "Kargo {hiz} geldi, ürün {durum}. {iade}",
    "Malzeme kalitesi {kalite_kotu}. {pisman}",
    "Destek ekibi {destek}. {sikayet}",
    "Ürün {yanlis}, {sorun}. {pisman}",
]
neu_templates = [
    "Ürün {orta}, işimi görüyor ama {daha_iyi}.",
    "Genel olarak {idare}, {kararsiz}.",
    "Teslimat {standart}, paketleme {normal}.",
    "Beklediğim kadar {hizli_degil} ama {kullanilabilir}.",
]

choices = {
    "hiz": ["çok hızlı", "zamanında", "geç", "biraz geç"],
    "kalite": ["çok kaliteli", "beklediğimden iyi", "gayet iyi", "harika"],
    "paket": ["çok iyi", "özenli", "sağlam", "düzgün"],
    "tavsiye": ["Kesinlikle tavsiye ederim.", "Tekrar alırım.", "5/5.", "Çok memnun kaldım."],
    "memnuniyet": ["Memnun kaldım.", "Sorunsuzdu.", "Gayet başarılı.", "Beklentimi karşıladı."],
    "performans": ["Performansı harika", "Hızlı çalışıyor", "Stabil", "Beklentimin üstünde"],
    "fiyatperf": ["fiyat/performans mükemmel", "parasını hak ediyor", "fiyatı uygun", "iyi bir tercih"],

    "durum": ["hasarlı geldi", "arızalı çıktı", "eksik geldi", "kalitesiz çıktı"],
    "iade": ["İade süreci çok zordu.", "İade ettim.", "Değişim istedim.", "Hayal kırıklığı."],
    "kalite_kotu": ["düşük", "kötü", "vasat", "berbat"],
    "pisman": ["Pişman oldum.", "Tavsiye etmiyorum.", "Bir daha almam.", "Çok kötü deneyim."],
    "destek": ["ulaşılmazdı", "geç döndü", "çözüm sunmadı", "hiç yardımcı olmadı"],
    "sikayet": ["Şikayetçiyim.", "Memnun değilim.", "Çözülmedi.", "Zaman kaybı."],
    "yanlis": ["anlatıldığı gibi değil", "beklediğim gibi değil", "yanlış ürün geldi", "uyumsuz çıktı"],
    "sorun": ["sürekli hata verdi", "çalışmadı", "performansı kötü", "problem çıkardı"],

    "orta": ["ortalama", "fena değil", "idare eder", "ne iyi ne kötü"],
    "daha_iyi": ["daha iyisi olabilir", "geliştirilebilir", "daha iyi beklerdim", "ortalamanın üstüne çıkmıyor"],
    "idare": ["idare eder", "orta seviye", "standart", "normal"],
    "kararsiz": ["kararsız kaldım", "çok emin değilim", "beklentim karışık", "deneyim orta"],
    "standart": ["standarttı", "normaldi", "beklendiği gibiydi", "orta halliydi"],
    "normal": ["normaldi", "eh işte", "standarttı", "vasattı"],
    "hizli_degil": ["hızlı değildi", "çok hızlı değildi", "beklediğim kadar hızlı değildi", "ortalama hızdaydı"],
    "kullanilabilir": ["kullanılabilir", "iş görür", "idare eder", "çok da kötü değil"],
}

def gen_one(template: str) -> str:
    # template içinde geçen placeholder’ları doldur
    text = template
    for k, vals in choices.items():
        text = text.replace("{"+k+"}", random.choice(vals))
    return text

def gen_rows(n_per_class: int = 400):
    rows = []
    for _ in range(n_per_class):
        rows.append({"review_text": gen_one(random.choice(pos_templates)), "label": "Olumlu"})
        rows.append({"review_text": gen_one(random.choice(neg_templates)), "label": "Olumsuz"})
        rows.append({"review_text": gen_one(random.choice(neu_templates)), "label": "Nötr"})
    return rows

rows = gen_rows(n_per_class=600)  # 600*3 = 1800 satır
df = pd.DataFrame(rows).sample(frac=1, random_state=42).reset_index(drop=True)
len(df), df.head()

split_column = "split"
df[split_column] = "TRAIN"
df.loc[df.index % 10 == 0, split_column] = "VALIDATE"  # %10
df.loc[df.index % 10 == 1, split_column] = "TEST"      # %10

df[split_column].value_counts()

## 3) CSV’yi GCS’ye yükle

In [ ]:
from google.cloud import storage

local_csv = "data/reviews_tabular_with_split.csv"
df.to_csv(local_csv, index=False, encoding="utf-8")

storage_client = storage.Client(project=project_id)
bucket = storage_client.bucket(bucket_name)

blob_path = f"{gcs_prefix}/{local_csv}"
blob = bucket.blob(blob_path)
blob.upload_from_filename(local_csv, content_type="text/csv")

gcs_uri = f"gs://{bucket_name}/{blob_path}"
print("✅ Uploaded:", gcs_uri)


## 4) Tabular Dataset oluştur + import

In [ ]:
dataset = aiplatform.TabularDataset.create(display_name=dataset_display_name, gcs_source=[gcs_uri])
print("✅ Dataset:", dataset.resource_name)

dataset.create(gcs_source=[gcs_uri])
print("✅ Import started. (Birkaç dakika sürebilir)")


## 5) AutoML Tabular Classification Training — Predefined Split

Burada `predefined_split_column_name="split"` kullanıyoruz.


In [ ]:
training_job = aiplatform.AutoMLTabularTrainingJob(
    display_name=model_display_name,
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

trained_model = training_job.run(
    dataset=dataset,
    target_column=target_column,
    predefined_split_column_name=split_column,
    model_display_name=model_display_name,
    sync=True,
)

print("✅ Model:", trained_model.resource_name)


## 6) Endpoint Deploy

In [ ]:
endpoint = trained_model.deploy(
    machine_type="n1-standard-2",
    sync=True,
)

print("✅ Endpoint:", endpoint.resource_name)


## 7) Predict

In [ ]:
test_texts = [
    "Kargo aşırı hızlıydı, ürün de çok kaliteli.",
    "Hiç memnun kalmadım, arızalı geldi ve kimse ilgilenmedi.",
    "Fena değil, işimi görüyor ama daha iyisini beklerdim."
]

instances = [{text_feature_column: t} for t in test_texts]
pred = endpoint.predict(instances=instances)

for i, t in enumerate(test_texts):
    print("\nYorum:", t)
    print("Raw prediction:", pred.predictions[i])


## 8) Cleanup (ÖNERİLİR)

In [ ]:
endpoint.delete(force=True)
# trained_model.delete()
# dataset.delete()

